# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 21 2022 1:36PM,249175,16,8675490,Sartorius Lab Products and Service,Released
1,Oct 21 2022 1:36PM,249175,16,8675498,Sartorius Lab Products and Service,Released
2,Oct 21 2022 1:27PM,249174,10,SONSB0001995,"Nextsource Biotechnology, LLC",Released
3,Oct 21 2022 1:17PM,249173,19,TR-RIH-10252022,"GCH Granules USA, Inc.",Released
4,Oct 21 2022 12:58PM,249171,10,Enova-10855,Emerginnova,Released
5,Oct 21 2022 12:57PM,249170,10,SO94426,"Senseonics, Incorporated",Released
6,Oct 21 2022 12:54PM,249166,10,0086096608,ISDIN Corporation,Released
7,Oct 21 2022 12:54PM,249166,10,0086096612,ISDIN Corporation,Released
8,Oct 21 2022 12:54PM,249166,10,0086096613,ISDIN Corporation,Released
9,Oct 21 2022 12:54PM,249166,10,0086096614,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,249170,Released,1
16,249171,Released,1
17,249173,Released,1
18,249174,Released,1
19,249175,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249170,NaN,NaN,1.0
249171,NaN,NaN,1.0
249173,NaN,NaN,1.0
249174,NaN,NaN,1.0
249175,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249088,0.0,0.0,1.0
249122,5.0,7.0,18.0
249132,10.0,1.0,3.0
249139,0.0,0.0,1.0
249146,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249088,0,0,1
249122,5,7,18
249132,10,1,3
249139,0,0,1
249146,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249088,0,0,1
1,249122,5,7,18
2,249132,10,1,3
3,249139,0,0,1
4,249146,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249088,,,1
1,249122,5,7,18
2,249132,10,1,3
3,249139,,,1
4,249146,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 21 2022 1:36PM,249175,16,Sartorius Lab Products and Service
2,Oct 21 2022 1:27PM,249174,10,"Nextsource Biotechnology, LLC"
3,Oct 21 2022 1:17PM,249173,19,"GCH Granules USA, Inc."
4,Oct 21 2022 12:58PM,249171,10,Emerginnova
5,Oct 21 2022 12:57PM,249170,10,"Senseonics, Incorporated"
6,Oct 21 2022 12:54PM,249166,10,ISDIN Corporation
62,Oct 21 2022 12:54PM,249169,10,ISDIN Corporation
73,Oct 21 2022 12:27PM,249157,19,"AdvaGen Pharma, Ltd"
76,Oct 21 2022 12:26PM,249156,10,Eywa Pharma Inc.
77,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 21 2022 1:36PM,249175,16,Sartorius Lab Products and Service,,,2
1,Oct 21 2022 1:27PM,249174,10,"Nextsource Biotechnology, LLC",,,1
2,Oct 21 2022 1:17PM,249173,19,"GCH Granules USA, Inc.",,,1
3,Oct 21 2022 12:58PM,249171,10,Emerginnova,,,1
4,Oct 21 2022 12:57PM,249170,10,"Senseonics, Incorporated",,,1
5,Oct 21 2022 12:54PM,249166,10,ISDIN Corporation,,,56
6,Oct 21 2022 12:54PM,249169,10,ISDIN Corporation,,,11
7,Oct 21 2022 12:27PM,249157,19,"AdvaGen Pharma, Ltd",,,3
8,Oct 21 2022 12:26PM,249156,10,Eywa Pharma Inc.,,,1
9,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 1:36PM,249175,16,Sartorius Lab Products and Service,2,,
1,Oct 21 2022 1:27PM,249174,10,"Nextsource Biotechnology, LLC",1,,
2,Oct 21 2022 1:17PM,249173,19,"GCH Granules USA, Inc.",1,,
3,Oct 21 2022 12:58PM,249171,10,Emerginnova,1,,
4,Oct 21 2022 12:57PM,249170,10,"Senseonics, Incorporated",1,,
5,Oct 21 2022 12:54PM,249166,10,ISDIN Corporation,56,,
6,Oct 21 2022 12:54PM,249169,10,ISDIN Corporation,11,,
7,Oct 21 2022 12:27PM,249157,19,"AdvaGen Pharma, Ltd",3,,
8,Oct 21 2022 12:26PM,249156,10,Eywa Pharma Inc.,1,,
9,Oct 21 2022 11:29AM,249148,19,VITABIOTICS USA INC.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 1:36PM,249175,16,Sartorius Lab Products and Service,2,,
1,Oct 21 2022 1:27PM,249174,10,"Nextsource Biotechnology, LLC",1,,
2,Oct 21 2022 1:17PM,249173,19,"GCH Granules USA, Inc.",1,,
3,Oct 21 2022 12:58PM,249171,10,Emerginnova,1,,
4,Oct 21 2022 12:57PM,249170,10,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 1:36PM,249175,16,Sartorius Lab Products and Service,2.0,NaN,NaN
1,Oct 21 2022 1:27PM,249174,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Oct 21 2022 1:17PM,249173,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Oct 21 2022 12:58PM,249171,10,Emerginnova,1.0,NaN,NaN
4,Oct 21 2022 12:57PM,249170,10,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 21 2022 1:36PM,249175,16,Sartorius Lab Products and Service,2.0,0.0,0.0
1,Oct 21 2022 1:27PM,249174,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Oct 21 2022 1:17PM,249173,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Oct 21 2022 12:58PM,249171,10,Emerginnova,1.0,0.0,0.0
4,Oct 21 2022 12:57PM,249170,10,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2242406,92.0,9.0,15.0
16,249175,2.0,0.0,0.0
19,996625,6.0,0.0,0.0
21,498227,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2242406,92.0,9.0,15.0
1,16,249175,2.0,0.0,0.0
2,19,996625,6.0,0.0,0.0
3,21,498227,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,92.0,9.0,15.0
1,16,2.0,0.0,0.0
2,19,6.0,0.0,0.0
3,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,92.0
1,16,Released,2.0
2,19,Released,6.0
3,21,Released,2.0
4,10,Executing,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21
Status,,,,
Completed,15.0,0.0,0.0,0.0
Executing,9.0,0.0,0.0,0.0
Released,92.0,2.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21
0,Completed,15.0,0.0,0.0,0.0
1,Executing,9.0,0.0,0.0,0.0
2,Released,92.0,2.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21
0,Completed,15.0,0.0,0.0,0.0
1,Executing,9.0,0.0,0.0,0.0
2,Released,92.0,2.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()